In [4]:
%load_ext autoreload
%autoreload 2

import misc
import config
import numpy as np
import blueprint
import tensorflow as tf
import visualization

layers_indexes = config.get_layers_indexes(4)
config = config.Config()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2022-09-25 01:52:45.883846: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-25 01:52:45.888714: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-25 01:52:45.889194: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-25 01:52:45.889778: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [5]:
genome = np.array([[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,2]])
module_genome = misc.module_convert(genome[0], layers_indexes)
module_graph = misc.get_graph(module_genome)


In [ ]:
genome_conv = misc.genome_convert(genome[0], config)
dot = visualization.visualize_genome_main(genome_conv, config)



In [ ]:
print(genome_conv)

In [6]:
b = blueprint.Blueprint(genome[0], config)


In [7]:
b.model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

In [8]:
b.model.fit(config.ds_train,
                    epochs=10,
                    use_multiprocessing=False,
                    validation_data=config.ds_test,
                    batch_size=128)

Epoch 1/10


/home/miluxd/anaconda3/envs/tf_new/lib/python3.9/site-packages/keras/backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
2022-09-25 01:53:06.630045: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8202


1563/1563 [==============================] - 23s 13ms/step - loss: 1.5771 - sparse_categorical_accuracy: 0.4120 - val_loss: 1.2764 - val_sparse_categorical_accuracy: 0.5437
Epoch 2/10
1563/1563 [==============================] - 26s 17ms/step - loss: 1.1004 - sparse_categorical_accuracy: 0.6013 - val_loss: 1.0637 - val_sparse_categorical_accuracy: 0.6275
Epoch 3/10
1563/1563 [==============================] - 24s 15ms/step - loss: 0.9028 - sparse_categorical_accuracy: 0.6806 - val_loss: 1.0114 - val_sparse_categorical_accuracy: 0.6456
Epoch 4/10
1563/1563 [==============================] - 27s 17ms/step - loss: 0.7569 - sparse_categorical_accuracy: 0.7340 - val_loss: 0.9989 - val_sparse_categorical_accuracy: 0.6659
Epoch 5/10
1563/1563 [==============================] - 28s 18ms/step - loss: 0.6307 - sparse_categorical_accuracy: 0.7796 - val_loss: 1.0525 - val_sparse_categorical_accuracy: 0.6672
Epoch 6/10
1563/1563 [==============================] - 29s 19ms/step - loss: 0.5325 - spar

In [ ]:
b.model.evaluate(config.ds_test)

In [ ]:
b.evaluate_model()

In [ ]:
%load_ext tensorboard
import tensorflow as tf


In [ ]:
log_dir = ""
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
b.model.fit(config.ds_train,
          epochs=5, 
          validation_data=(config.ds_test), 
          callbacks=[tensorboard_callback])


In [ ]:
input = tf.keras.Input(shape=config.input_shape, dtype=config.dtype)
last_layer = tf.keras.layers.Flatten()(input)
layer = tf.keras.layers.Dense(units=16,activation='relu',dtype=tf.float32)
last_layer = layer(last_layer)


In [ ]:
output = tf.keras.layers.Dense(units=config.out_units, 
                        name='out_dense_layer',
                        activation=config.out_activation,
                        dtype=config.dtype)(last_layer)

In [ ]:
model2 = tf.keras.Model(input, output)
model2.compile(optimizer=config.optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
model2.fit(config.ds_train,
                    epochs=5,
                    use_multiprocessing=True,
                    batch_size=config.batch_size)

In [ ]:
model2.summary()

In [ ]:
model3 = tf.keras.models.Sequential([
  # tf.keras.Input(shape=config.input_shape, dtype=config.dtype),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
model3.compile(config.optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

model3.fit(
    config.ds_train,
    epochs=5,
    validation_data=config.ds_test,
)


In [ ]:
model4 = tf.keras.models.Sequential([
  tf.keras.Input(shape=config.input_shape, dtype=config.dtype),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
model4.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
model4.fit(
    config.ds_train,
    epochs=5,
    validation_data=config.ds_test,
)